## Import of libraries & dataset

### Import Libraries

In [1]:
!pip install pyspark
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, TimestampType, DoubleType
import pandas as pd
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import isnan, isnull, col, sum, when, input_file_name, udf, to_timestamp, datediff
from pyspark.ml.feature import Bucketizer
import pandas as pd
import numpy as np
from pandas import Series
from numpy import var, mean, sqrt
import seaborn as sb
import matplotlib.pyplot as plt

In [2]:
# Define a schema for the CSV files
schema = StructType([
    StructField("ddl_case_id object", StringType(), True),
    StructField("year", StringType(), True),
    StructField("state_code", StringType(), True),
    StructField("dist_code", StringType(), True),
    StructField("court_no", StringType(), True),
    StructField("cino", StringType(), True),
    StructField("judge_position", StringType(), True),
    StructField("female_defendant", StringType(), True),
    StructField("female_petitioner", StringType(), True),
    StructField("female_adv_def", StringType(), True),
    StructField("female_adv_pet", StringType(), True),
    StructField("type_name", IntegerType(), True),
    StructField("purpose_name", StringType(), True),
    StructField("disp_name", StringType(), True),
    StructField("date_of_filing", StringType(), True),
    StructField("date_of_decision", StringType(), True),
    StructField("date_first_list", StringType(), True),
    StructField("date_last_list", StringType(), True),
    StructField("date_next_list", StringType(), True),
    StructField("act", StringType(), True),
    StructField("section", StringType(), True),
    StructField("billable_ipc", StringType(), True),
    StructField("number_sections_ipc", StringType(), True),
    StructField("criminal", StringType(), True),
    StructField("ddl_filing_judge_id", StringType(), True),
    StructField("ddl_decision_judge_id", StringType(), True),
    StructField("female_judge_filing", StringType(), True),
    StructField("start_date_filing", StringType(), True),
    StructField("end_date_filing", StringType(), True),
    StructField("female_judge_decision", StringType(), True),
    StructField("start_date_decision", StringType(), True),
    StructField("end_date_decision", StringType(), True),
    StructField("judgediff", StringType(), True),
    StructField("pendency_decision_filing", StringType(), True),
    StructField("pendency_first_filing", StringType(), True),
    StructField("pendency_decision_first", StringType(), True),
    StructField("pendency_next_last", StringType(), True),
    StructField("pendency_decision_last", StringType(), True),
    StructField("pendency_decision_next", StringType(), True),
    StructField("pendency_decision_filing_categories", StringType(), True),
    StructField("pendency_first_filing_categories", StringType(), True),
    StructField("filing_judge_tenure",StringType(), True),
    StructField("decision_judge_tenure", StringType(), True),
])

### Import of Cases 2010

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
MAX_MEMORY = '45G'
conf = pyspark.SparkConf().setMaster("local[*]") \
        .set('spark.executor.heartbeatInterval', 10000) \
        .set('spark.network.timeout', 10000) \
        .set("spark.core.connection.ack.wait.timeout", "3600") \
        .set("spark.executor.memory", MAX_MEMORY) \
        .set("spark.driver.memory", MAX_MEMORY)

# Create a Spark session
spark = SparkSession.builder.appName("CSV Addition").config(conf=conf).getOrCreate()

In [5]:

# Define a list of file paths for the CSV files
file_paths = ['/content/drive/MyDrive/Law/cases/2010/cases_2010_final.csv',
              '/content/drive/MyDrive/Law/cases/2011/cases_2011_final.csv',
              '/content/drive/MyDrive/Law/cases/2012/cases_2012_final.csv',
              '/content/drive/MyDrive/Law/cases/2013/cases_2013_final.csv',
              '/content/drive/MyDrive/Law/cases/2014/cases_2014_final.csv',
              '/content/drive/MyDrive/Law/cases/2015/cases_2015_final.csv',
              '/content/drive/MyDrive/Law/cases/2016/cases_2016_final.csv',
              '/content/drive/MyDrive/Law/cases/2017/cases_2017_final.csv',
              '/content/drive/MyDrive/Law/cases/2018/cases_2018_final.csv']

# Read the CSV files into a single dataframe
df_cases_orig = spark.read.csv(
    file_paths,
    schema=schema,
    header=True, # set to False if there's no header
    timestampFormat="yyyy-MM-dd'T'HH:mm:ss", # specify the format for timestamp columns
    #mode="FAILFAST" # fail fast if any errors occur while reading the CSV files
    mode="PERMISIVE" # fail fast if any errors occur while reading the CSV files
)

# Display the combined dataframe
df_cases_orig.show(5)

+--------------------+----+----------+---------+--------+----------------+--------------------+----------------+-----------------+--------------+--------------+---------+------------+---------+--------------------+--------------------+--------------------+--------------------+--------------------+-------+---------+------------+-------------------+--------+-------------------+---------------------+-------------------+-----------------+---------------+---------------------+-------------------+-----------------+---------+------------------------+---------------------+-----------------------+------------------+----------------------+----------------------+-----------------------------------+--------------------------------+-------------------+---------------------+
|  ddl_case_id object|year|state_code|dist_code|court_no|            cino|      judge_position|female_defendant|female_petitioner|female_adv_def|female_adv_pet|type_name|purpose_name|disp_name|      date_of_filing|    date_of_de

In [6]:
# Define a list of file paths for the CSV files
file_paths = ['/content/drive/MyDrive/Law/religion/cases_religion_2018.csv',
              '/content/drive/MyDrive/Law/religion/cases_religion_2017.csv',
              '/content/drive/MyDrive/Law/religion/cases_religion_2016.csv',
              '/content/drive/MyDrive/Law/religion/cases_religion_2015.csv',
              '/content/drive/MyDrive/Law/religion/cases_religion_2014.csv',
              '/content/drive/MyDrive/Law/religion/cases_religion_2013.csv',
              '/content/drive/MyDrive/Law/religion/cases_religion_2012.csv',
              '/content/drive/MyDrive/Law/religion/cases_religion_2011.csv',
              '/content/drive/MyDrive/Law/religion/cases_religion_2010.csv']

# Read the CSV files into a single dataframe
df_cases_orig_religion = spark.read.csv(
    file_paths,
    #schema=schema,
    header=True, # set to False if there's no header
    #timestampFormat="yyyy-MM-dd'T'HH:mm:ss", # specify the format for timestamp columns
    #mode="FAILFAST" # fail fast if any errors occur while reading the CSV files
    mode="PERMISIVE" # fail fast if any errors occur while reading the CSV files
)

# Display the combined dataframe
df_cases_orig_religion.show(5)

+--------------------+----------------+-----------------+--------------+--------------+
|         ddl_case_id|muslim_defendant|muslim_petitioner|muslim_adv_def|muslim_adv_pet|
+--------------------+----------------+-----------------+--------------+--------------+
|01-01-01-20190000...|     0 nonmuslim|      0 nonmuslim|         -9999|             0|
|01-01-01-20190000...|     0 nonmuslim|      0 nonmuslim|         -9999|             0|
|01-01-01-20190000...|     0 nonmuslim|      0 nonmuslim|         -9999|             0|
|01-01-01-20190000...|     0 nonmuslim|      0 nonmuslim|         -9999|             0|
|01-01-01-20190000...|   -9998 unclear|      0 nonmuslim|         -9999|             0|
+--------------------+----------------+-----------------+--------------+--------------+
only showing top 5 rows



In [7]:
df_cases_orig =  df_cases_orig.withColumn("act", col("act").cast("integer"))
df_cases_orig =  df_cases_orig.withColumn("section", col("section").cast("integer"))
#column_name ="section"
#null_count = df_cases_orig.select(sum(col(column_name).isNull().cast("int"))).collect()[0][0]
#print(null_count)

In [8]:
total_rows = df_cases_orig.count()
print(total_rows)

79313266


In [9]:
total_rows_religion = df_cases_orig_religion.count()
print(total_rows_religion)

80935944


In [10]:
df_cases_orig.dtypes

[('ddl_case_id object', 'string'),
 ('year', 'string'),
 ('state_code', 'string'),
 ('dist_code', 'string'),
 ('court_no', 'string'),
 ('cino', 'string'),
 ('judge_position', 'string'),
 ('female_defendant', 'string'),
 ('female_petitioner', 'string'),
 ('female_adv_def', 'string'),
 ('female_adv_pet', 'string'),
 ('type_name', 'int'),
 ('purpose_name', 'string'),
 ('disp_name', 'string'),
 ('date_of_filing', 'string'),
 ('date_of_decision', 'string'),
 ('date_first_list', 'string'),
 ('date_last_list', 'string'),
 ('date_next_list', 'string'),
 ('act', 'int'),
 ('section', 'int'),
 ('billable_ipc', 'string'),
 ('number_sections_ipc', 'string'),
 ('criminal', 'string'),
 ('ddl_filing_judge_id', 'string'),
 ('ddl_decision_judge_id', 'string'),
 ('female_judge_filing', 'string'),
 ('start_date_filing', 'string'),
 ('end_date_filing', 'string'),
 ('female_judge_decision', 'string'),
 ('start_date_decision', 'string'),
 ('end_date_decision', 'string'),
 ('judgediff', 'string'),
 ('pendency

In [11]:
old_header_name = "ddl_case_id object"
new_header_name = "ddl_case_id"

df_cases_orig = df_cases_orig.withColumnRenamed(old_header_name, new_header_name)

In [12]:
df_cases_orig  = df_cases_orig.join(df_cases_orig_religion, on="ddl_case_id", how="left")

In [13]:
df_cases_orig.show()

+--------------------+----+----------+---------+--------+----------------+--------------------+----------------+-----------------+--------------+--------------+---------+------------+---------+--------------------+--------------------+--------------------+--------------------+--------------------+-----+-------+------------+-------------------+--------+-------------------+---------------------+-------------------+-----------------+---------------+---------------------+-------------------+-----------------+---------+------------------------+---------------------+-----------------------+------------------+----------------------+----------------------+-----------------------------------+--------------------------------+-------------------+---------------------+----------------+-----------------+--------------+--------------+
|         ddl_case_id|year|state_code|dist_code|court_no|            cino|      judge_position|female_defendant|female_petitioner|female_adv_def|female_adv_pet|type_na

## Preprocessing & Feature Engineering

In [14]:
df_cases_orig.printSchema()

root
 |-- ddl_case_id: string (nullable = true)
 |-- year: string (nullable = true)
 |-- state_code: string (nullable = true)
 |-- dist_code: string (nullable = true)
 |-- court_no: string (nullable = true)
 |-- cino: string (nullable = true)
 |-- judge_position: string (nullable = true)
 |-- female_defendant: string (nullable = true)
 |-- female_petitioner: string (nullable = true)
 |-- female_adv_def: string (nullable = true)
 |-- female_adv_pet: string (nullable = true)
 |-- type_name: integer (nullable = true)
 |-- purpose_name: string (nullable = true)
 |-- disp_name: string (nullable = true)
 |-- date_of_filing: string (nullable = true)
 |-- date_of_decision: string (nullable = true)
 |-- date_first_list: string (nullable = true)
 |-- date_last_list: string (nullable = true)
 |-- date_next_list: string (nullable = true)
 |-- act: integer (nullable = true)
 |-- section: integer (nullable = true)
 |-- billable_ipc: string (nullable = true)
 |-- number_sections_ipc: string (nullable

In [15]:
from pyspark.sql import functions as F

# Assuming df_cases_orig is your Spark DataFrame
df_cases_orig = df_cases_orig.withColumn("court_details", F.concat_ws("_", "state_code", "dist_code", "court_no"))
df_cases_orig = df_cases_orig.withColumn("state_district", F.concat_ws("_", "state_code", "dist_code"))

In [ ]:
#df_cases_orig  = df_cases_orig [df_cases_orig ['pendency_decision_filing_categories'] != 'Undecided']

In [16]:
#print(df_cases_orig.count())
# filter all cases where the case is Undecided and disp_name is not 26
rows_to_remove_mask = (
    (df_cases_orig["pendency_decision_filing_categories"] == "Undecided") &
    ((df_cases_orig["disp_name"] != "26") & (df_cases_orig["disp_name"] != "27"))
)

# Create a copy of the original DataFrame and drop the rows to be removed
df_cases_orig = df_cases_orig[~rows_to_remove_mask]
# Create a copy of the original DataFrame and drop the rows to be removed
#print(df_cases_orig.count())

In [17]:
# create columns for binary classification problem
mapping_multi = {'upto 1 year': 'ontime' ,'1 to 2 years': 'ontime', '2 to 3 years': 'delayed','3 to 4 years': 'delayed','4 to 5 years': 'delayed', '5 to 6 years': 'delayed','6 to 7 years': 'delayed', '7 to 8 years': 'delayed','8 to 9 years': 'delayed', '9 to 10 years': 'delayed',  '10 to 11 years': 'delayed', 'Undecided': 'delayed'}
def map_decision(decision):
    return mapping_multi.get(decision, None)

# Register the UDF
map_decision_udf = F.udf(map_decision)

# Use the UDF to create a new column "pendency_model_multiyear"
df_cases_orig = df_cases_orig.withColumn("pendency_model_multiyear",
                                         map_decision_udf("pendency_decision_filing_categories"))

In [18]:
total_rows = df_cases_orig.count()
print(total_rows)

79293276


In [ ]:
### Drop cases impacted by COVID lockdowns in India
from pyspark.sql.functions import to_timestamp, col, lit


df_cases_orig = df_cases_orig.withColumn("start_date_filing", to_timestamp("start_date_filing", 'yyyy-MM-dd'))
df_cases_orig = df_cases_orig.withColumn("date_of_decision", to_timestamp("start_date_filing", 'yyyy-MM-dd'))

# Define the cutoff dates
cutoff_filing_date = lit("2018-03-25").cast("timestamp")
cutoff_decision_date = lit("2020-03-25").cast("timestamp")

# Apply the filtering logic
df_cases_orig = df_cases_orig.filter(
    ~((col("start_date_filing") > cutoff_filing_date) & col("date_of_decision").isNull())
)
df_cases_orig = df_cases_orig.filter(col("date_of_decision") <= cutoff_decision_date)

In [ ]:
total_rows = df_cases_orig.count()
print(total_rows)

10785356


In [ ]:
# Assuming you have a Spark DataFrame called 'df_cases2010'
df_cases_orig = df_cases_orig.withColumn("date_of_filing", to_timestamp("date_of_filing", "yyyy-MM-dd HH:mm:ssXXX"))

df_cases_orig = df_cases_orig.withColumn("filing_judge_tenure_at_filing", when(col("start_date_filing").isNull() | col("date_of_filing").isNull(), None)
                                        .otherwise(
                                            (datediff(col("date_of_filing"), col("start_date_filing"))/30).cast(IntegerType())
                                        ))

In [ ]:
unique_jt_values = df_cases_orig.select("filing_judge_tenure_at_filing").distinct()
#totcount = unique_jt_values.count()
#unique_jt_values.show(totcount, truncate=False)

In [ ]:
# Assuming you have a Spark DataFrame called 'df_cases2010'
df_cases_orig = df_cases_orig.withColumn("start_date_decision", to_timestamp("start_date_decision", 'yyyy-MM-dd'))

df_cases_orig = df_cases_orig.withColumn("decision_judge_tenure_at_decision", when(col("start_date_decision").isNull() | col("date_of_decision").isNull(), None)
                                        .otherwise(
                                            (datediff(col("date_of_decision"), col("start_date_decision"))/30).cast(IntegerType())
                                        ))

In [ ]:
unique_jt_values = df_cases_orig.select("decision_judge_tenure_at_decision").distinct()
#totcount = unique_jt_values.count()
#unique_jt_values.show(totcount, truncate=False)

In [ ]:
#print(df_cases_orig.count())

79293276


We will drop following features

* ddl_filing_judge_id
* ddl_decision_judge_id
* start_date_filing
* end_date_filing
* start_date_decision
* end_date_decision

The following are dropped as these dates are converted into categorical variables as periods

* disp_name

The following are dropped as they are just identifiers

* ddl_case_id object
* purpose_name

These variables are not included as deemed not important*

* pendency_decision_first
* pendency_decision_last
* pendency_decision_next
* pendency_next_last

Drop these as these are feature engineered for target variables

* pendency_decision_filing_categories

Drop these as these are feature engineered and used

* dist_code
* court_no


In [ ]:
columns_to_drop = ['ddl_filing_judge_id','ddl_decision_judge_id','start_date_filing','end_date_filing','start_date_decision','end_date_decision',\
                   'ddl_case_id','purpose_name','date_first_list','date_last_list','date_next_list',\
                   'pendency_decision_first','pendency_decision_last', 'pendency_decision_next','pendency_next_last',\
                   'dist_code','court_no']

# Create a copy of the dataframe and start preparing the same for prediction modelling
df_cases_modified_merge = df_cases_orig.drop(*columns_to_drop)

In [ ]:
df_cases_modified_merge.printSchema()

root
 |-- year: string (nullable = true)
 |-- state_code: string (nullable = true)
 |-- cino: string (nullable = true)
 |-- judge_position: string (nullable = true)
 |-- female_defendant: string (nullable = true)
 |-- female_petitioner: string (nullable = true)
 |-- female_adv_def: string (nullable = true)
 |-- female_adv_pet: string (nullable = true)
 |-- type_name: integer (nullable = true)
 |-- disp_name: string (nullable = true)
 |-- date_of_filing: timestamp (nullable = true)
 |-- date_of_decision: timestamp (nullable = true)
 |-- act: integer (nullable = true)
 |-- section: integer (nullable = true)
 |-- billable_ipc: string (nullable = true)
 |-- number_sections_ipc: string (nullable = true)
 |-- criminal: string (nullable = true)
 |-- female_judge_filing: string (nullable = true)
 |-- female_judge_decision: string (nullable = true)
 |-- judgediff: string (nullable = true)
 |-- pendency_decision_filing: string (nullable = true)
 |-- pendency_first_filing: string (nullable = true

## Merge data from modified Act, Section and Judge Position File

In [ ]:
df_act = spark.read.csv("/content/drive/MyDrive/Law/keys/act_key.csv", header=True, inferSchema=True)
df_section = spark.read.csv("/content/drive/MyDrive/Law/keys/section_key.csv", header=True, inferSchema=True)
df_type = spark.read.csv("/content/drive/MyDrive/Law/keys/type_name_key.csv", header=True, inferSchema=True)

In [ ]:
df_act  = df_act.withColumnRenamed("count", "countact")
df_act = df_act.dropna(how='all')
df_act.show(5)


+--------------------+--------+----+
|               act_s|countact| act|
+--------------------+--------+----+
|                NULL| 11282.0|NULL|
|                   '| 11160.0|   1|
|                  ''| 20887.0|   2|
|'Tamil Nadu City ...|   371.0|   3|
|(BOMBAY) NATIONAL...|     2.0|   4|
+--------------------+--------+----+
only showing top 5 rows



In [ ]:
df_act.printSchema()

root
 |-- act_s: string (nullable = true)
 |-- countact: double (nullable = true)
 |-- act: integer (nullable = true)



In [ ]:
df_section=  df_section.withColumnRenamed("count", "countsection")
df_section = df_section.dropna(how='all')
df_section.show(5)

+--------------------+------------+-------+
|           section_s|countsection|section|
+--------------------+------------+-------+
|                NULL|     4834909|   NULL|
|!#11bofHindumarri...|           1|      1|
|              !3 (B)|           1|      2|
|          !3(1)(i-a)|           1|      3|
|               !3(3)|           1|      4|
+--------------------+------------+-------+
only showing top 5 rows



In [ ]:
df_section.printSchema()

root
 |-- section_s: string (nullable = true)
 |-- countsection: integer (nullable = true)
 |-- section: integer (nullable = true)



In [ ]:
df_type = df_type.drop("count")
df_type.show(5)

+----+---------+------------------+
|year|type_name|       type_name_s|
+----+---------+------------------+
|2010|        1|          (m) t.s.|
|2010|        2|              (sc)|
|2010|        3|        ..mact 166|
|2010|        4|04 complaint cases|
|2010|        5|  04 criminal case|
+----+---------+------------------+
only showing top 5 rows



In [ ]:
df_cases_modified_mergedact  = df_cases_modified_merge.join(df_act, on="act", how="left")

In [ ]:
df_cases_modified_mergedact.show(5)

+----+----+----------+----------------+--------------------+----------------+-----------------+--------------+--------------+---------+---------+-------------------+-------------------+-------+------------+-------------------+--------+-------------------+---------------------+---------+------------------------+---------------------+-----------------------------------+--------------------------------+-------------------+---------------------+----------------+-----------------+--------------+--------------+-------------+--------------+------------------------+-----------------------------+---------------------------------+--------------------+---------+
| act|year|state_code|            cino|      judge_position|female_defendant|female_petitioner|female_adv_def|female_adv_pet|type_name|disp_name|     date_of_filing|   date_of_decision|section|billable_ipc|number_sections_ipc|criminal|female_judge_filing|female_judge_decision|judgediff|pendency_decision_filing|pendency_first_filing|penden

In [ ]:
print(df_cases_modified_mergedact.count())

10785356


In [ ]:
df_cases_modified_mergedactsection  = df_cases_modified_mergedact.join(df_section,on="section", how="left")

In [ ]:
df_cases_modified_mergedactsection.show(5)

+-------+----+----+----------+----------------+--------------------+----------------+-----------------+--------------+--------------+---------+---------+-------------------+-------------------+------------+-------------------+--------+-------------------+---------------------+---------+------------------------+---------------------+-----------------------------------+--------------------------------+-------------------+---------------------+----------------+-----------------+--------------+--------------+-------------+--------------+------------------------+-----------------------------+---------------------------------+--------------------+---------+---------+------------+
|section| act|year|state_code|            cino|      judge_position|female_defendant|female_petitioner|female_adv_def|female_adv_pet|type_name|disp_name|     date_of_filing|   date_of_decision|billable_ipc|number_sections_ipc|criminal|female_judge_filing|female_judge_decision|judgediff|pendency_decision_filing|pende

In [ ]:
df_cases_modified_mergedactsectiontype = df_cases_modified_mergedactsection.join(df_type, on = ["year","type_name"],  how="left")

In [ ]:
print(df_cases_modified_mergedactsection.count())

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/lib/python3.10/dist-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: ignored

In [ ]:
df_judge = spark.read.csv("/content/drive/MyDrive/Law/judges_clean/judge_position_key.csv", header=True, inferSchema=True)

In [ ]:
def merge_judge_position(df):
    df  = df.join(df_judge,on="judge_position", how="left")
    df  = df.drop("judge_position")

    return df

In [ ]:
df_cases_modified_mergedactsectionjp = merge_judge_position(df_cases_modified_mergedactsectiontype)

In [ ]:
df_cases_modified_mergedactsectionjp.printSchema()

root
 |-- year: string (nullable = true)
 |-- type_name: integer (nullable = true)
 |-- section: integer (nullable = true)
 |-- act: integer (nullable = true)
 |-- state_code: string (nullable = true)
 |-- cino: string (nullable = true)
 |-- female_defendant: string (nullable = true)
 |-- female_petitioner: string (nullable = true)
 |-- female_adv_def: string (nullable = true)
 |-- female_adv_pet: string (nullable = true)
 |-- disp_name: string (nullable = true)
 |-- date_of_filing: timestamp (nullable = true)
 |-- date_of_decision: timestamp (nullable = true)
 |-- billable_ipc: string (nullable = true)
 |-- number_sections_ipc: string (nullable = true)
 |-- criminal: string (nullable = true)
 |-- female_judge_filing: string (nullable = true)
 |-- female_judge_decision: string (nullable = true)
 |-- judgediff: string (nullable = true)
 |-- pendency_decision_filing: string (nullable = true)
 |-- pendency_first_filing: string (nullable = true)
 |-- pendency_decision_filing_categories: st

In [ ]:
#Transform number_sections_ipc
from pyspark.ml.feature import Bucketizer
from pyspark.sql.types import DoubleType
bins = [-1, 1, 2, 3, 4, 5, 10, 15, 20, 25, 30, 35, 40, float('inf')]
labelsipc = ['zero', 'one', 'two', 'three', 'four', '5 to 10', '10 to 15', '15 to 20',
          '20 to 25', '25 to 30', '30 to 35', '35 to 40', '40 plus']

# Define a UDF to map indices to labels
def map_index_to_label_ipc(index):
    if index is not None:
        return labelsipc[index]
    else:
        return "NA"

# Register the UDF
map_index_udf_ipc = udf(map_index_to_label_ipc, StringType())

# Assuming 'pendency_decision_filing' is the column you want to bin
# You can also cast it to DoubleType to match the bin boundaries
df_cases_modified_mergedactsectionjp = df_cases_modified_mergedactsectionjp.withColumn("number_sections_ipc", col("number_sections_ipc").cast(DoubleType()))

# Use Bucketizer to create a new column with the labels
bucketizer = Bucketizer(splits=bins, inputCol="number_sections_ipc", outputCol="number_sections_ipc_temp")
df_cases_modified_mergedactsectionjp = bucketizer.transform(df_cases_modified_mergedactsectionjp)

df_cases_modified_mergedactsectionjp = df_cases_modified_mergedactsectionjp.withColumn("number_sections_ipc_temp", col("number_sections_ipc_temp").cast(IntegerType()))

# Add a new column "label_values" to the DataFrame using the UDF
df_cases_modified_mergedactsectionjp = df_cases_modified_mergedactsectionjp.withColumn("number_sections_ipc_category", map_index_udf_ipc(df_cases_modified_mergedactsectionjp["number_sections_ipc_temp"]))

In [ ]:
df_cases_sample = df_cases_modified_mergedactsectionjp.select("number_sections_ipc", "number_sections_ipc_category")
df_cases_sample = df_cases_sample.filter(~(isnan(col("number_sections_ipc")) | isnull(col("number_sections_ipc"))))
df_cases_sample.show()

+-------------------+----------------------------+
|number_sections_ipc|number_sections_ipc_category|
+-------------------+----------------------------+
|                3.0|                       three|
|                5.0|                     5 to 10|
|                2.0|                         two|
|                2.0|                         two|
|                3.0|                       three|
|                2.0|                         two|
|                3.0|                       three|
|                1.0|                         one|
|                1.0|                         one|
|                3.0|                       three|
|                3.0|                       three|
|                4.0|                        four|
|                4.0|                        four|
|                5.0|                     5 to 10|
|                2.0|                         two|
|                2.0|                         two|
|                4.0|          

In [ ]:
#Transform pendency
bins = [-1, 0, 1, 3, 6, 9, 12, 24, 36, 48, 60, 72, 84, 96, 108, 120, float('inf')]
labelspf = ['upto 1 month', '1 month', '1 to 3 months', '3 to 6 months', '6 month to 9 month' , '9 month to 1 year', '1 to 2 years', '2 to 3 years', '3 to 4 years', '4 to 5 years',
          '5 to 6 years', '6 to 7 years', '7 to 8 years', '8 to 9 years', '9 to 10 years',
          '10 year +']

# Define a UDF to map indices to labels
def map_index_to_label_pf(index):
    if index is not None:
        return labelspf[index]
    else:
        return "NA"

# Register the UDF
map_index_udf_pf = udf(map_index_to_label_pf, StringType())

# Assuming 'pendency_decision_filing' is the column you want to bin
# You can also cast it to DoubleType to match the bin boundaries
df_cases_modified_mergedactsectionjp = df_cases_modified_mergedactsectionjp.withColumn("pendency_first_filing", col("pendency_first_filing").cast(DoubleType()))

# Use Bucketizer to create a new column with the labels
bucketizer = Bucketizer(splits=bins, inputCol="pendency_first_filing", outputCol="pendency_first_filing_temp")
df_cases_modified_mergedactsectionjp = bucketizer.transform(df_cases_modified_mergedactsectionjp)

df_cases_modified_mergedactsectionjp = df_cases_modified_mergedactsectionjp.withColumn("pendency_first_filing_temp", col("pendency_first_filing_temp").cast(IntegerType()))

# Add a new column "label_values" to the DataFrame using the UDF
df_cases_modified_mergedactsectionjp = df_cases_modified_mergedactsectionjp.withColumn("pendency_first_filing_category", map_index_udf_pf(df_cases_modified_mergedactsectionjp["pendency_first_filing_temp"]))


In [ ]:
df_cases_sample = df_cases_modified_mergedactsectionjp.select("pendency_first_filing", "pendency_first_filing_temp", "pendency_first_filing_category")
df_cases_sample = df_cases_sample.filter(~(isnan(col("pendency_first_filing")) | isnull(col("pendency_first_filing"))))
df_cases_sample.show()

+---------------------+--------------------------+------------------------------+
|pendency_first_filing|pendency_first_filing_temp|pendency_first_filing_category|
+---------------------+--------------------------+------------------------------+
|                  0.0|                         1|                       1 month|
|                  0.0|                         1|                       1 month|
|                  0.0|                         1|                       1 month|
|                  1.0|                         2|                 1 to 3 months|
|                  1.0|                         2|                 1 to 3 months|
|                  0.0|                         1|                       1 month|
|                  0.0|                         1|                       1 month|
|                  2.0|                         2|                 1 to 3 months|
|                  1.0|                         2|                 1 to 3 months|
|               

In [ ]:
#Transform filling_judge_tenure_at_filing
from pyspark.ml.feature import Bucketizer
from pyspark.sql.types import DoubleType
from pyspark.sql.functions import lit, expr
bins = [-1, 1, 3, 6, 12, 24, 36, 48, 60, 72, 84, 96, 108, 120, 180, float('inf')]
labels = ['upto 1 months', '1 to 3 months', '3 to 6 months', '6 month to 1 year', '1 to 2 years', '2 to 3 years', '3 to 4 years', '4 to 5 years',
          '5 to 6 years', '6 to 7 years', '7 to 8 years', '8 to 9 years', '9 to 10 years',
          '10 to 15 years', '15 years plus']

# Define a UDF to map indices to labels
def map_index_to_label(index):
    if index is not None:
        return labels[index]
    else:
        return "NA"

# Register the UDF
map_index_udf = udf(map_index_to_label, StringType())

# Assuming 'pendency_decision_filing' is the column you want to bin
# You can also cast it to DoubleType to match the bin boundaries
df_cases_modified_mergedactsectionjp = df_cases_modified_mergedactsectionjp.withColumn("filing_judge_tenure_at_filing", col("filing_judge_tenure_at_filing").cast(DoubleType()))

# Use Bucketizer to create a new column with the labels
bucketizer = Bucketizer(splits=bins, inputCol="filing_judge_tenure_at_filing", outputCol="filing_judge_tenure_temp")
df_cases_modified_mergedactsectionjp = bucketizer.transform(df_cases_modified_mergedactsectionjp)

df_cases_modified_mergedactsectionjp = df_cases_modified_mergedactsectionjp.withColumn("filing_judge_tenure_temp", col("filing_judge_tenure_temp").cast(IntegerType()))

# Add a new column "label_values" to the DataFrame using the UDF
df_cases_modified_mergedactsectionjp = df_cases_modified_mergedactsectionjp.withColumn("filing_judge_tenure_at_filing_category", map_index_udf(df_cases_modified_mergedactsectionjp["filing_judge_tenure_temp"]))

In [ ]:
df_cases_sample = df_cases_modified_mergedactsectionjp.select("filing_judge_tenure_at_filing", "filing_judge_tenure_at_filing_category")
df_cases_sample = df_cases_sample.filter(~(isnan(col("filing_judge_tenure_at_filing")) | isnull(col("filing_judge_tenure_at_filing_category"))))
df_cases_sample.show()

+-----------------------------+--------------------------------------+
|filing_judge_tenure_at_filing|filing_judge_tenure_at_filing_category|
+-----------------------------+--------------------------------------+
|                         24.0|                          2 to 3 years|
|                          8.0|                     6 month to 1 year|
|                          1.0|                         1 to 3 months|
|                         19.0|                          1 to 2 years|
|                          3.0|                         3 to 6 months|
|                         14.0|                          1 to 2 years|
|                         25.0|                          2 to 3 years|
|                         24.0|                          2 to 3 years|
|                         24.0|                          2 to 3 years|
|                          9.0|                     6 month to 1 year|
|                         11.0|                     6 month to 1 year|
|     

In [ ]:
#Transform decision_judge_tenure_at_decision
from pyspark.ml.feature import Bucketizer
from pyspark.sql.types import DoubleType
from pyspark.sql.functions import lit, expr
bins = [-1, 1, 3, 6, 12, 24, 36, 48, 60, 72, 84, 96, 108, 120, 180, float('inf')]
labels = ['upto 1 months', '1 to 3 months', '3 to 6 months', '6 month to 1 year', '1 to 2 years', '2 to 3 years', '3 to 4 years', '4 to 5 years',
          '5 to 6 years', '6 to 7 years', '7 to 8 years', '8 to 9 years', '9 to 10 years',
          '10 to 15 years', '15 years plus']

# Define a UDF to map indices to labels
def map_index_to_label(index):
    if index is not None:
        return labels[index]
    else:
        return "NA"

# Register the UDF
map_index_udf = udf(map_index_to_label, StringType())

# Assuming 'pendency_decision_filing' is the column you want to bin
# You can also cast it to DoubleType to match the bin boundaries
df_cases_modified_mergedactsectionjp = df_cases_modified_mergedactsectionjp.withColumn("decision_judge_tenure_at_decision", col("decision_judge_tenure_at_decision").cast(DoubleType()))

# Use Bucketizer to create a new column with the labels
bucketizer = Bucketizer(splits=bins, inputCol="decision_judge_tenure_at_decision", outputCol="decision_judge_tenure_temp")
df_cases_modified_mergedactsectionjp = bucketizer.transform(df_cases_modified_mergedactsectionjp)

df_cases_modified_mergedactsectionjp = df_cases_modified_mergedactsectionjp.withColumn("decision_judge_tenure_temp", col("decision_judge_tenure_temp").cast(IntegerType()))

# Add a new column "label_values" to the DataFrame using the UDF
df_cases_modified_mergedactsectionjp = df_cases_modified_mergedactsectionjp.withColumn("decision_judge_tenure_at_decision_category", map_index_udf(df_cases_modified_mergedactsectionjp["decision_judge_tenure_temp"]))

IllegalArgumentException: requirement failed: Column decision_judge_tenure_temp already exists.

In [ ]:
df_cases_sample = df_cases_modified_mergedactsectionjp.select("decision_judge_tenure_at_decision", "decision_judge_tenure_at_decision_category")
df_cases_sample = df_cases_sample.filter(~(isnan(col("decision_judge_tenure_at_decision")) | isnull(col("decision_judge_tenure_at_decision_category"))))
#df_cases_sample.show()

Py4JJavaError: An error occurred while calling o599.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 3 in stage 174.0 failed 1 times, most recent failure: Lost task 3.0 in stage 174.0 (TID 3454) (595cdbb20a3d executor driver): org.apache.spark.SparkException: [FAILED_EXECUTE_UDF] Failed to execute user defined function (`bucketizer_0 (Bucketizer$$Lambda$4427/0x00007fb8e486d040)`: (double) => double).
	at org.apache.spark.sql.errors.QueryExecutionErrors$.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala:198)
	at org.apache.spark.sql.errors.QueryExecutionErrors.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$SpecificMutableProjection.ScalaUDF_0$(Unknown Source)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$SpecificMutableProjection.If_0$(Unknown Source)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$SpecificMutableProjection.apply

In [ ]:
df_cases_modified_mergedactsectionjp.printSchema()

root
 |-- year: string (nullable = true)
 |-- type_name: integer (nullable = true)
 |-- section: integer (nullable = true)
 |-- act: integer (nullable = true)
 |-- state_code: string (nullable = true)
 |-- cino: string (nullable = true)
 |-- female_defendant: string (nullable = true)
 |-- female_petitioner: string (nullable = true)
 |-- female_adv_def: string (nullable = true)
 |-- female_adv_pet: string (nullable = true)
 |-- disp_name: string (nullable = true)
 |-- date_of_filing: timestamp (nullable = true)
 |-- date_of_decision: timestamp (nullable = true)
 |-- billable_ipc: string (nullable = true)
 |-- number_sections_ipc: double (nullable = true)
 |-- criminal: string (nullable = true)
 |-- female_judge_filing: string (nullable = true)
 |-- female_judge_decision: string (nullable = true)
 |-- judgediff: string (nullable = true)
 |-- pendency_decision_filing: string (nullable = true)
 |-- pendency_first_filing: double (nullable = true)
 |-- pendency_decision_filing_categories: st

In [ ]:
columns_to_drop_postmerge = ['pendency_first_filing_category', 'disp_name','type_name','number_sections_ipc',
                             'filing_judge_tenure','filing_judge_tenure_temp','pendency_first_filing_temp','number_sections_ipc_temp']

#columns_to_drop_postmerge = ['billable_ipc','type_name',','type_name','act','section','cino','disp_name','number_sections_ipc','pendency_decision_filling','pendency_first_filing','filing_judge_tenure','pendency_first_filing_categories','filing_judge_tenure_temp','pendency_first_filing_temp','number_sections_ipc_temp']
df_cases_modified_final = df_cases_modified_mergedactsectionjp.drop(*columns_to_drop_postmerge)

In [ ]:
df_cases_modified_final.printSchema()

root
 |-- year: string (nullable = true)
 |-- section: integer (nullable = true)
 |-- act: integer (nullable = true)
 |-- state_code: string (nullable = true)
 |-- cino: string (nullable = true)
 |-- female_defendant: string (nullable = true)
 |-- female_petitioner: string (nullable = true)
 |-- female_adv_def: string (nullable = true)
 |-- female_adv_pet: string (nullable = true)
 |-- date_of_filing: timestamp (nullable = true)
 |-- date_of_decision: timestamp (nullable = true)
 |-- billable_ipc: string (nullable = true)
 |-- criminal: string (nullable = true)
 |-- female_judge_filing: string (nullable = true)
 |-- female_judge_decision: string (nullable = true)
 |-- judgediff: string (nullable = true)
 |-- pendency_decision_filing: string (nullable = true)
 |-- pendency_first_filing: double (nullable = true)
 |-- pendency_decision_filing_categories: string (nullable = true)
 |-- pendency_first_filing_categories: string (nullable = true)
 |-- decision_judge_tenure: string (nullable = 

In [ ]:
columns_to_drop_postmerge = ['section','act','cino', 'date_of_filing', 'judge_position', 'date_of_decision','pendency_first_filing_categories', 'decision_judge_tenure']

df_cases_modified_final = df_cases_modified_final.drop(*columns_to_drop_postmerge)

In [ ]:
df_cases_modified_final.printSchema()

root
 |-- year: string (nullable = true)
 |-- state_code: string (nullable = true)
 |-- female_defendant: string (nullable = true)
 |-- female_petitioner: string (nullable = true)
 |-- female_adv_def: string (nullable = true)
 |-- female_adv_pet: string (nullable = true)
 |-- billable_ipc: string (nullable = true)
 |-- criminal: string (nullable = true)
 |-- female_judge_filing: string (nullable = true)
 |-- female_judge_decision: string (nullable = true)
 |-- judgediff: string (nullable = true)
 |-- pendency_decision_filing: string (nullable = true)
 |-- pendency_first_filing: double (nullable = true)
 |-- pendency_decision_filing_categories: string (nullable = true)
 |-- muslim_defendant: string (nullable = true)
 |-- muslim_petitioner: string (nullable = true)
 |-- muslim_adv_def: string (nullable = true)
 |-- muslim_adv_pet: string (nullable = true)
 |-- court_details: string (nullable = false)
 |-- state_district: string (nullable = false)
 |-- pendency_model_multiyear: string (nu

In [ ]:
columns_to_drop_postmerge = ['pendency_decision_filing_categories','decision_judge_tenure','countact', 'count','decision_judge_tenure_temp', 'decision_judge_tenure_at_decision_category','filing_judge_tenure_at_filing_category']

df_cases_modified = df_cases_modified_final.drop(*columns_to_drop_postmerge)

In [ ]:
df_cases_modified.printSchema()

root
 |-- year: string (nullable = true)
 |-- state_code: string (nullable = true)
 |-- female_defendant: string (nullable = true)
 |-- female_petitioner: string (nullable = true)
 |-- female_adv_def: string (nullable = true)
 |-- female_adv_pet: string (nullable = true)
 |-- billable_ipc: string (nullable = true)
 |-- criminal: string (nullable = true)
 |-- female_judge_filing: string (nullable = true)
 |-- female_judge_decision: string (nullable = true)
 |-- judgediff: string (nullable = true)
 |-- pendency_decision_filing: string (nullable = true)
 |-- pendency_first_filing: double (nullable = true)
 |-- muslim_defendant: string (nullable = true)
 |-- muslim_petitioner: string (nullable = true)
 |-- muslim_adv_def: string (nullable = true)
 |-- muslim_adv_pet: string (nullable = true)
 |-- court_details: string (nullable = false)
 |-- state_district: string (nullable = false)
 |-- pendency_model_multiyear: string (nullable = true)
 |-- filing_judge_tenure_at_filing: double (nullable

In [ ]:
df_cases_modified= df_cases_modified.fillna("NA")

In [ ]:
df_cases_modified = df_cases_modified.filter(col("criminal") == 1)


In [ ]:
df_cases_modified = df_cases_modified.filter(((col("female_adv_def") == '1')|(col("female_adv_def") == '0')))
df_cases_modified = df_cases_modified.filter(((col("female_adv_pet") == '0')|(col("female_adv_pet") == '1')))
df_cases_modified = df_cases_modified.filter(((col("muslim_adv_def") == '1')|(col("muslim_adv_def") == '0')))
df_cases_modified = df_cases_modified.filter(((col("muslim_adv_pet") == '1')|(col("muslim_adv_pet") == '0')))

In [ ]:
df_cases_modified = df_cases_modified.filter(((col("female_petitioner") == '1 female')|(col("female_petitioner") == '0 male')))
df_cases_modified = df_cases_modified.filter(((col("female_defendant") == '0')|(col("female_defendant") == '1')))
df_cases_modified = df_cases_modified.filter(((col("muslim_defendant") == '0 nonmuslim')|(col("muslim_defendant") == '1 muslim')))
df_cases_modified = df_cases_modified.filter(((col("muslim_petitioner") == '0 nonmuslim')|(col("muslim_petitioner") == '1 muslim')))

In [ ]:
df_cases_modified = df_cases_modified.filter(((col("female_judge_filing") == '1 female')|(col("female_judge_filing") == '0 nonfemale')))
df_cases_modified = df_cases_modified.filter(((col("female_judge_decision") == '1 female')|(col("female_judge_decision") == '0 nonfemale')))
#df_cases_modified = df_cases_modified.filter(((col("judgediff") == '1.0')|(col("judgediff") == '0.0')))

In [ ]:
print(df_cases_modified.count())


82026


In [ ]:
df_cases_modified.show(20)

+----+----------+----------------+-----------------+--------------+--------------+------------+--------+-------------------+---------------------+---------+------------------------+---------------------+----------------+-----------------+--------------+--------------+-------------+--------------+------------------------+-----------------------------+---------------------------------+--------------------+---------+------------+-----------+--------------------+----------------------------+
|year|state_code|female_defendant|female_petitioner|female_adv_def|female_adv_pet|billable_ipc|criminal|female_judge_filing|female_judge_decision|judgediff|pendency_decision_filing|pendency_first_filing|muslim_defendant|muslim_petitioner|muslim_adv_def|muslim_adv_pet|court_details|state_district|pendency_model_multiyear|filing_judge_tenure_at_filing|decision_judge_tenure_at_decision|               act_s|section_s|countsection|type_name_s|    judge_position_s|number_sections_ipc_category|
+----+--------

In [ ]:
df_cases_modified.printSchema()

root
 |-- year: string (nullable = false)
 |-- state_code: string (nullable = false)
 |-- female_defendant: string (nullable = false)
 |-- female_petitioner: string (nullable = false)
 |-- female_adv_def: string (nullable = false)
 |-- female_adv_pet: string (nullable = false)
 |-- billable_ipc: string (nullable = false)
 |-- criminal: string (nullable = false)
 |-- female_judge_filing: string (nullable = false)
 |-- female_judge_decision: string (nullable = false)
 |-- judgediff: string (nullable = false)
 |-- pendency_decision_filing: string (nullable = false)
 |-- pendency_first_filing: double (nullable = true)
 |-- muslim_defendant: string (nullable = false)
 |-- muslim_petitioner: string (nullable = false)
 |-- muslim_adv_def: string (nullable = false)
 |-- muslim_adv_pet: string (nullable = false)
 |-- court_details: string (nullable = false)
 |-- state_district: string (nullable = false)
 |-- pendency_model_multiyear: string (nullable = false)
 |-- filing_judge_tenure_at_filing:

In [ ]:
columns_to_drop = ['countsection','pendency_decision_filing','criminal']

# Create a copy of the dataframe and start preparing the same for prediction modelling
df_cases_modified = df_cases_modified.drop(*columns_to_drop)

In [ ]:
df_cases_modified.printSchema()

root
 |-- year: string (nullable = false)
 |-- state_code: string (nullable = false)
 |-- female_defendant: string (nullable = false)
 |-- female_petitioner: string (nullable = false)
 |-- female_adv_def: string (nullable = false)
 |-- female_adv_pet: string (nullable = false)
 |-- billable_ipc: string (nullable = false)
 |-- female_judge_filing: string (nullable = false)
 |-- female_judge_decision: string (nullable = false)
 |-- judgediff: string (nullable = false)
 |-- pendency_first_filing: double (nullable = true)
 |-- muslim_defendant: string (nullable = false)
 |-- muslim_petitioner: string (nullable = false)
 |-- muslim_adv_def: string (nullable = false)
 |-- muslim_adv_pet: string (nullable = false)
 |-- court_details: string (nullable = false)
 |-- state_district: string (nullable = false)
 |-- pendency_model_multiyear: string (nullable = false)
 |-- filing_judge_tenure_at_filing: double (nullable = true)
 |-- decision_judge_tenure_at_decision: double (nullable = true)
 |-- ac

In [ ]:
state_code_df = df_cases_modified.groupBy("year").count()
totcount = state_code_df.count()
state_code_df.show(totcount, truncate=False)
print(totcount)

+----+-----+
|year|count|
+----+-----+
|2016|13452|
|2012|4143 |
|2017|13904|
|2014|12682|
|2013|6461 |
|2018|12104|
|2011|2767 |
|2015|14780|
|2010|1733 |
+----+-----+

9


In [ ]:
state_code_df = df_cases_modified.groupBy("type_name_s").count()
totcount = state_code_df.count()
state_code_df.show(totcount, truncate=False)
print(totcount)

+-------------------------------+-----+
|type_name_s                    |count|
+-------------------------------+-----+
|cc juve                        |7    |
|execution cases                |2    |
|cc                             |1654 |
|mjc                            |2    |
|cr                             |19   |
|indian penal code (ipc)        |5    |
|civil m.a.                     |2    |
|sessions case  t ii            |4    |
|rct                            |320  |
|complaint cases lct            |39   |
|crlmp                          |71   |
|anticipatory bail              |48   |
|ba                             |358  |
|crl.a                          |40   |
|criminal misscellaneous        |15   |
|crl. case                      |2    |
|spl.sc                         |21   |
|crl .misc.                     |10   |
|cr. rev.                       |1    |
|criminal appeal                |71   |
|cr en                          |18   |
|cri.appeal                     |329  |


In [ ]:
jp_df = df_cases_modified.groupBy("pendency_model_multiyear").count()
totcount = jp_df.count()
jp_df.show(totcount, truncate=False)
print(totcount)

+------------------------+-----+
|pendency_model_multiyear|count|
+------------------------+-----+
|ontime                  |52005|
|delayed                 |30021|
+------------------------+-----+

2


In [ ]:
jp_df = df_cases_modified.groupBy("pendency_first_filing").count()
totcount = jp_df.count()
jp_df.show(totcount, truncate=False)
print(totcount)

+---------------------+-----+
|pendency_first_filing|count|
+---------------------+-----+
|8.0                  |635  |
|0.0                  |41616|
|7.0                  |792  |
|49.0                 |21   |
|29.0                 |85   |
|42.0                 |27   |
|44.0                 |18   |
|NULL                 |1327 |
|62.0                 |6    |
|35.0                 |61   |
|18.0                 |273  |
|1.0                  |17773|
|104.0                |1    |
|39.0                 |21   |
|94.0                 |2    |
|34.0                 |59   |
|37.0                 |53   |
|25.0                 |140  |
|36.0                 |71   |
|41.0                 |23   |
|4.0                  |1591 |
|23.0                 |183  |
|50.0                 |9    |
|56.0                 |11   |
|31.0                 |89   |
|11.0                 |490  |
|21.0                 |191  |
|72.0                 |4    |
|14.0                 |398  |
|63.0                 |3    |
|22.0     

In [ ]:
jp_df = df_cases_modified.groupBy("filing_judge_tenure_at_filing").count()
totcount = jp_df.count()
jp_df.show(totcount, truncate=False)
print(totcount)

+-----------------------------+-----+
|filing_judge_tenure_at_filing|count|
+-----------------------------+-----+
|67.0                         |83   |
|8.0                          |2741 |
|70.0                         |70   |
|0.0                          |6303 |
|69.0                         |108  |
|7.0                          |3356 |
|88.0                         |16   |
|49.0                         |139  |
|101.0                        |3    |
|98.0                         |9    |
|29.0                         |799  |
|64.0                         |80   |
|75.0                         |11   |
|47.0                         |130  |
|42.0                         |181  |
|44.0                         |133  |
|35.0                         |498  |
|96.0                         |9    |
|62.0                         |85   |
|18.0                         |1727 |
|120.0                        |6    |
|1.0                          |5092 |
|39.0                         |270  |
|104.0      

In [ ]:
jp_df = df_cases_modified.groupBy("decision_judge_tenure_at_decision").count()
totcount = jp_df.count()
jp_df.show(totcount, truncate=False)
print(totcount)

+---------------------------------+-----+
|decision_judge_tenure_at_decision|count|
+---------------------------------+-----+
|0.0                              |47040|
|-140.0                           |24   |
|-74.0                            |203  |
|-58.0                            |183  |
|-81.0                            |163  |
|-52.0                            |255  |
|-59.0                            |125  |
|-167.0                           |3    |
|-80.0                            |50   |
|-148.0                           |12   |
|-5.0                             |227  |
|-18.0                            |338  |
|-29.0                            |295  |
|-50.0                            |265  |
|-127.0                           |18   |
|-42.0                            |368  |
|-73.0                            |1110 |
|-2.0                             |205  |
|-26.0                            |484  |
|-1.0                             |125  |
|-106.0                           

In [ ]:
state_code_df = df_cases_modified.groupBy("state_code").count()
totcount = state_code_df.count()
state_code_df.show(totcount, truncate=False)
print(totcount)

+----------+-----+
|state_code|count|
+----------+-----+
|7         |1261 |
|15        |1253 |
|11        |4123 |
|29        |3986 |
|3         |4411 |
|30        |70   |
|8         |1120 |
|22        |6761 |
|16        |508  |
|5         |896  |
|18        |1067 |
|27        |35   |
|17        |7725 |
|26        |76   |
|6         |673  |
|23        |5397 |
|25        |187  |
|9         |2336 |
|1         |9973 |
|20        |477  |
|10        |8490 |
|4         |236  |
|12        |465  |
|13        |5869 |
|14        |5795 |
|2         |8814 |
|19        |2    |
|24        |15   |
|21        |5    |
+----------+-----+

29


In [ ]:
jp_df = df_cases_modified.groupBy("state_district").count()
totcount = jp_df.count()
jp_df.show(totcount, truncate=False)
print(totcount)

+--------------+-----+
|state_district|count|
+--------------+-----+
|23_15         |68   |
|23_27         |47   |
|11_11         |456  |
|10_32         |80   |
|22_13         |365  |
|5_4           |735  |
|13_58         |182  |
|1_30          |91   |
|2_7           |460  |
|8_19          |90   |
|2_23          |140  |
|14_11         |196  |
|11_21         |847  |
|29_7          |78   |
|1_21          |203  |
|5_11          |40   |
|1_33          |83   |
|2_19          |74   |
|22_15         |459  |
|23_44         |110  |
|16_19         |176  |
|10_27         |89   |
|23_16         |20   |
|18_19         |14   |
|10_18         |411  |
|17_15         |138  |
|13_17         |43   |
|3_19          |808  |
|16_13         |8    |
|13_33         |85   |
|17_32         |15   |
|13_19         |29   |
|8_26          |34   |
|14_1          |305  |
|22_4          |30   |
|29_2          |1259 |
|14_16         |281  |
|18_6          |80   |
|9_39          |13   |
|18_3          |11   |
|17_21     

In [ ]:
jp_df = df_cases_modified.groupBy("court_details").count()
totcount = jp_df.count()
jp_df.show(totcount, truncate=False)
print(totcount)

+-------------+-----+
|court_details|count|
+-------------+-----+
|10_13_4      |10   |
|9_27_7       |1    |
|23_49_2      |398  |
|23_1_4       |14   |
|10_28_14     |13   |
|3_14_1       |15   |
|3_26_20      |49   |
|22_14_1      |50   |
|23_39_1      |33   |
|11_29_1      |45   |
|3_18_1       |19   |
|16_7_7       |10   |
|1_8_3        |3    |
|17_14_2      |34   |
|9_11_6       |4    |
|3_19_1       |46   |
|15_7_2       |109  |
|9_12_10      |15   |
|2_4_29       |117  |
|14_17_1      |81   |
|23_28_1      |17   |
|17_28_3      |17   |
|13_67_5      |34   |
|11_26_3      |195  |
|6_24_3       |37   |
|15_9_3       |11   |
|7_4_4        |20   |
|17_16_4      |34   |
|17_17_8      |6    |
|16_10_5      |12   |
|2_17_17      |1    |
|14_4_12      |30   |
|20_1_4       |27   |
|9_3_11       |1    |
|1_13_1       |44   |
|2_4_1        |12   |
|15_15_9      |3    |
|8_11_1       |121  |
|25_1_6       |8    |
|14_11_1      |196  |
|18_17_1      |13   |
|9_23_6       |56   |
|17_26_8  

In [ ]:
jp_df = df_cases_modified.groupBy("female_judge_filing").count()
totcount = jp_df.count()
jp_df.show(totcount, truncate=False)
print(totcount)

+-------------------+-----+
|female_judge_filing|count|
+-------------------+-----+
|1 female           |19030|
|0 nonfemale        |62996|
+-------------------+-----+

2


In [ ]:
jp_df = df_cases_modified.groupBy("female_judge_decision").count()
totcount = jp_df.count()
jp_df.show(totcount, truncate=False)
print(totcount)

+---------------------+-----+
|female_judge_decision|count|
+---------------------+-----+
|1 female             |20045|
|0 nonfemale          |61981|
+---------------------+-----+

2


In [ ]:
jp_df = df_cases_modified.groupBy("judgediff").count()
totcount = jp_df.count()
jp_df.show(totcount, truncate=False)
print(totcount)

+---------+-----+
|judgediff|count|
+---------+-----+
|1.0      |35056|
|0.0      |46970|
+---------+-----+

2


In [ ]:
jp_df = df_cases_modified.groupBy("billable_ipc").count()
totcount = jp_df.count()
jp_df.show(totcount, truncate=False)
print(totcount)

+-------------------------+-----+
|billable_ipc             |count|
+-------------------------+-----+
|NA                       |61026|
|not applicable to section|1102 |
|unclear, need to check   |93   |
|bailable                 |10194|
|non-bailable             |8849 |
|depends                  |762  |
+-------------------------+-----+

6


In [ ]:
jp_df = df_cases_modified.groupBy("number_sections_ipc_category").count()
totcount = jp_df.count()
jp_df.show(totcount, truncate=False)
print(totcount)

+----------------------------+-----+
|number_sections_ipc_category|count|
+----------------------------+-----+
|10 to 15                    |197  |
|two                         |4153 |
|5 to 10                     |3961 |
|NA                          |53624|
|four                        |3135 |
|one                         |13297|
|three                       |3601 |
|15 to 20                    |39   |
|20 to 25                    |16   |
|25 to 30                    |3    |
+----------------------------+-----+

10


In [ ]:
jp_df = df_cases_modified.groupBy("judge_position_s").count()
totcount = jp_df.count()
jp_df.show(totcount, truncate=False)
print(totcount)

+------------------------------------------------+-----+
|judge_position_s                                |count|
+------------------------------------------------+-----+
|cantonment court                                |31   |
|additional district judge                       |1186 |
|Civil Judge Senior Division                     |1668 |
|3-additional district judge                     |38   |
|city civil and sessions court                   |1556 |
|District And Sessions Court                     |40432|
|8-additional district judge                     |43   |
|porahat division                                |5    |
|District Civil Court                            |6968 |
|subdivisional judicial magistrate               |494  |
|Judicial Magistrate Court                       |3906 |
|2-additional district judge                     |268  |
|outlying court                                  |25   |
|ndps                                            |18   |
|taluka court                  

In [ ]:
jp_df = df_cases_modified.groupBy("female_defendant").count()
jp_df.show()

+----------------+-----+
|female_defendant|count|
+----------------+-----+
|               0|68467|
|               1|13559|
+----------------+-----+



In [ ]:
jp_df = df_cases_modified.groupBy("female_petitioner").count()
jp_df.show()

+-----------------+-----+
|female_petitioner|count|
+-----------------+-----+
|         1 female|31277|
|           0 male|50749|
+-----------------+-----+



In [ ]:
jp_df = df_cases_modified.groupBy("female_adv_def").count()
jp_df.show()

+--------------+-----+
|female_adv_def|count|
+--------------+-----+
|             0|76131|
|             1| 5895|
+--------------+-----+



In [ ]:
jp_df = df_cases_modified.groupBy("female_adv_pet").count()
jp_df.show()

+--------------+-----+
|female_adv_pet|count|
+--------------+-----+
|             0|74972|
|             1| 7054|
+--------------+-----+



In [ ]:
jp_df = df_cases_modified.groupBy("muslim_defendant").count()
jp_df.show()

+----------------+-----+
|muslim_defendant|count|
+----------------+-----+
|        1 muslim| 7454|
|     0 nonmuslim|74572|
+----------------+-----+



In [ ]:
jp_df = df_cases_modified.groupBy("muslim_petitioner").count()
jp_df.show()

+-----------------+-----+
|muslim_petitioner|count|
+-----------------+-----+
|         1 muslim| 7339|
|      0 nonmuslim|74687|
+-----------------+-----+



In [ ]:
jp_df = df_cases_modified.groupBy("muslim_adv_def").count()
jp_df.show()

+--------------+-----+
|muslim_adv_def|count|
+--------------+-----+
|             0|74789|
|             1| 7237|
+--------------+-----+



In [ ]:
jp_df = df_cases_modified.groupBy("muslim_adv_pet").count()
jp_df.show()

+--------------+-----+
|muslim_adv_pet|count|
+--------------+-----+
|             0|74979|
|             1| 7047|
+--------------+-----+



In [ ]:
jp_df = df_cases_modified.groupBy("act_s").count()
totcount = jp_df.count()
jp_df.show(totcount, truncate=False)
print(totcount)

+--------------------------+-----+
|act_s                     |count|
+--------------------------+-----+
|Code of Criminal Procedure|53624|
|The Indian Penal Code     |28402|
+--------------------------+-----+

2


In [ ]:
jp_df = df_cases_modified.groupBy("section_s").count()
totcount = jp_df.count()
jp_df.show(totcount, truncate=False)
print(totcount)

+---------+-----+
|section_s|count|
+---------+-----+
|7        |21   |
|15       |17   |
|232      |16   |
|317      |6    |
|428      |1    |
|424      |1    |
|434      |4    |
|308      |19   |
|34       |27   |
|52       |5    |
|183      |1    |
|441      |3    |
|223      |1    |
|202      |2    |
|.        |3799 |
|219      |3    |
|131      |6    |
|130      |1    |
|381      |43   |
|328      |35   |
|201      |15   |
|344      |3    |
|104      |2    |
|229      |1    |
|90       |1    |
|429      |23   |
|23       |19   |
|25       |35   |
|270      |4    |
|176      |1    |
|82       |4    |
|461      |4    |
|363      |57   |
|366      |553  |
|500      |175  |
|427      |233  |
|24       |34   |
|480      |1    |
|134      |1    |
|127      |1038 |
|49       |299  |
|306      |206  |
|21       |33   |
|264      |2    |
|469      |2    |
|161      |80   |
|321      |1    |
|367      |6    |
|499      |9    |
|368      |2    |
|451      |74   |
|124      |1    |
|272      

In [ ]:
df_cases_modified.show()

+----+----------+----------------+-----------------+--------------+--------------+------------+-------------------+---------------------+---------+---------------------+----------------+-----------------+--------------+--------------+-------------+--------------+------------------------+-----------------------------+---------------------------------+--------------------+---------+-----------+--------------------+----------------------------+
|year|state_code|female_defendant|female_petitioner|female_adv_def|female_adv_pet|billable_ipc|female_judge_filing|female_judge_decision|judgediff|pendency_first_filing|muslim_defendant|muslim_petitioner|muslim_adv_def|muslim_adv_pet|court_details|state_district|pendency_model_multiyear|filing_judge_tenure_at_filing|decision_judge_tenure_at_decision|               act_s|section_s|type_name_s|    judge_position_s|number_sections_ipc_category|
+----+----------+----------------+-----------------+--------------+--------------+------------+-------------

### Save as one file

In [ ]:
df_cases_modified.coalesce(1).write.csv("automl_2010to2018_final_criminal.csv", header=True, mode="overwrite")

In [ ]:
!cat /content/automl_2010to2018_final_criminal.csv/*.csv > /content/automl_2010to18_final_criminal.csv

In [ ]:
!cp /content/automl_2010to18_final_criminal.csv /content/drive/MyDrive/Law/Model1abData_2010to2018_merged_criminal_final.csv
